In [1]:
import json

with open('../data/submissions-2016-2020-with-failed.json') as json_f:
    submissions = json.load(json_f)['submissions']

In [37]:
import random

index = random.randint(0, len(submissions))

In [38]:
submissions[index]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Barf-Vader',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_3cwy8ucm',
 'author_patreon_flair': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1572741767,
 'domain': 'self.AmItheAsshole',
 'full_link': 'https://www.reddit.com/r/AmItheAsshole/comments/dqt9ud/am_i_the_asshole/',
 'gildings': {},
 'id': 'dqt9ud',
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 1,
 'num_crossposts': 0,
 'over_18': False,
 'parent_whitelist_status': 'all_ads',
 'permalink': '/r/

In [39]:
import yaml
from praw import Reddit


with open('auth.yaml') as yaml_f:
    auth_obj = yaml.full_load(yaml_f)


reddit = Reddit(
    username=auth_obj['username'],
    password=auth_obj['password'],
    client_id=auth_obj['client-id'],
    client_secret=auth_obj['client-secret'],
    user_agent='desktop:aita_prediction_bot:v0.0.0 (by /u/aita_prediction_bot)'
)

In [40]:
from praw.models import Submission

submission = Submission(reddit, url=submissions[index]['url'])

In [41]:
print(submission.selftext)

[removed]


In [42]:
submission.link_flair_text

In [36]:
from praw.models import MoreComments

submission.comment_sort = 'best'
submission.comment_limit = 5
for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    # Here you can fetch data off the comment.
    # For the sake of example, we're just printing the comment body.
    print(top_level_comment.body)

On the one hand, you really should have gotten explicit permission from every roommate before getting the cat. I'm allergic to cats and my quality of life would go way down if my roommate just decided to bring one home one day. On the other hand, you gave him a chance to say no and he just said "meh". I suspect you knew he didn't want you to get one but you did it anyway because he didn't actually SAY no, which is pretty assholey. The roommate is definitely an asshole for being so passive aggressive about everything.

I'd say both assholes, you for not caring about your roommate's desire to not have a cat, and him for being rude and passive aggressive about it.
First things first, he's an asshole for treating the cat in that way, even if he regrets its presence in the house.  But you're not in the IsMyFlatmateTheAsshole sub, so lets move on to your own culpabilities here.

While in theory your flatmate *could* have communicated his opposition when you were first getting the cat, at tha

## What do we need from each submission?

For training
- The text
- The flair (if it exists)
- The top n comments (set to 5 in this case)

For Metadata
- The title
- The url
- The score

Allowed 1 rq/s so we will add a 1s sleep after each request

In [84]:
import time
from tqdm import tqdm

n_comments = 5
raw_submissions = []
failed = []

for submission in tqdm(submissions):
    try:
        # check if it was already removed
        if not submission['selftext'] == '[removed]':
            submission_obj = Submission(reddit, url=submission['url'])
            submission_obj.comment_sort = 'best'
            submission_obj.comment_limit = 5
            comments = []
            for top_level_comment in submission_obj.comments:
                if isinstance(top_level_comment, MoreComments):
                    continue
                comments.append(top_level_comment.body)
            if submission_obj.selftext == '[deleted]' or submission_obj.selftext == '[removed]':
                desired_text = submission['selftext']
            else:
                desired_text = submission_obj.selftext
            
            raw_submission = {
                'text': desired_text,
                'flair': submission_obj.link_flair_text if submission_obj.link_flair_text else '',
                'comments': comments,
                'title': submission_obj.title,
                'url': submission_obj.url,
                'score': submission_obj.score
            }
            raw_submissions.append(raw_submission)
            
            time.sleep(2)
            if reddit.auth.limits['remaining'] < 2:
                time.sleep(int(reddit.auth.limits['reset_timestamp'] - time.time()))
        else:
            failed.append(submission)
    except Exception as e:
        failed.append(submission)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25614/25614 [13:30:36<00:00,  1.90s/it]


In [85]:
with open('../data/submissions-2016-2020-raw-reddit-api-data.json', 'w') as json_f:
    json.dump(
        {
            'submissions': raw_submissions
        },
        json_f
    )

In [86]:
with open('../data/FAILED-submissions-2016-2020-raw-reddit-api-data.json', 'w') as json_f:
    json.dump(
        {
            'submissions': failed
        },
        json_f
    )

In [87]:
len(raw_submissions)

20389